In [ ]:
import pandas as pd

In [ ]:
# Creating a destination path
file_path = "/content/restaurants_preprocessed.csv"

df_restaurants = pd.read_csv(file_path)

# 1 Cleaning dataset

## 1.1 Getting rid of old index columns

In [ ]:
df_restaurants.head()

## 1.2 Ensuring we are using consistent datatypes (boolean)

### 1.2.1 First we check the datatypes, unique values, and value counts

In [ ]:
df_restaurants.dtypes

In [ ]:
for column in df_restaurants.columns:
    unique_values = df_restaurants[column].unique()
    print(f"Unique values in '{column}':\n{unique_values}\n")


In [ ]:
# Display value counts for each column in the DataFrame
for column in df_restaurants.columns:
    print(f"Value counts for '{column}':")
    print(df_restaurants[column].value_counts(dropna=False))
    print("\n")

### 1.2.2 We will change string True/False to booleans, change is_open to boolean, and assume that 'none' = 'false'

In [ ]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler

# Assuming df_restaurants is your DataFrame

# Step 1: Convert 'is_open' to boolean
df_restaurants['is_open'] = df_restaurants['is_open'].astype(bool)

# Step 2: Clean up boolean columns
bool_columns = [
    'delivery', 'alcohol', 'bike_parking', 'credit_card', 'appointment_only',
    'caters', 'coat_check', 'dogs', 'drive_thru', 'good_for_kids',
    'good_for_groups', 'happy_hour', 'tv', 'outdoor_seating',
    'reservations', 'table_service', 'take_out', 'wheelchair'
]

# Replace 'True'/'False' strings with actual boolean values and None with np.nan
for column in bool_columns:
    df_restaurants[column] = df_restaurants[column].replace({'True': True, 'False': False, 'None': np.nan})

# Step 3: Clean the 'price_range' column and handle NaN
# Convert any remaining non-numeric values (e.g., 'False') to NaN in 'price_range'
df_restaurants['price_range'] = pd.to_numeric(df_restaurants['price_range'], errors='coerce')

# Apply KNN Imputer on 'price_range'
knn_imputer = KNNImputer(n_neighbors=5)
df_restaurants[['price_range']] = knn_imputer.fit_transform(df_restaurants[['price_range']])

# Round the imputed values to the nearest integer and clip to valid price levels (1-4)
df_restaurants['price_range'] = df_restaurants['price_range'].round().clip(1, 4)

# Step 4: Replace any remaining NaN values in boolean columns with False
df_restaurants[bool_columns] = df_restaurants[bool_columns].fillna(False)

# Step 5: Impute the most common value for 'open_on_weekend'
# Get the most common value (mode)
most_common_value = df_restaurants['open_on_weekend'].mode()[0]
df_restaurants['open_on_weekend'].fillna(most_common_value, inplace=True)

# Step 6: Standardize 'hours_per_week'
scaler = StandardScaler()
df_restaurants['hours_per_week'] = scaler.fit_transform(df_restaurants[['hours_per_week']])

# Final Check: Display the cleaned and transformed data
print(df_restaurants[['price_range', 'hours_per_week', 'open_on_weekend']].head())

# Optional: Check the distribution of 'open_on_weekend' after imputation
print("Value counts for 'open_on_weekend':")
print(df_restaurants['open_on_weekend'].value_counts())

Now we re-verify the value counts

In [ ]:
df_restaurants.dtypes

We should check distribution of review_score by review_count

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Assuming df_restaurants is your DataFrame

# Scatter plot to visualize the relationship between stars and review_count
plt.figure(figsize=(10, 6))
sns.scatterplot(x='stars', y='review_count', data=df_restaurants)
plt.title('Scatter Plot of Stars by Review Count')
plt.xlabel('Review Count')
plt.ylabel('Stars')
plt.show()

# Calculate the correlation between stars and review_count
correlation = df_restaurants['stars'].corr(df_restaurants['review_count'])
print(f'Correlation between stars and review_count: {correlation}')

In [ ]:
import plotly.express as px

# Create the histogram using Plotly
fig = px.histogram(df_restaurants, x='review_count', nbins=1000, title='Distribution of Review Count', marginal='rug')

# Customize the labels
fig.update_layout(
    xaxis_title='Review Count',
    yaxis_title='Frequency',
    title_x=0.5
)

# Show the plot
fig.show()

In [ ]:
# Save the DataFrame as a CSV file
df_restaurants.to_csv('df_restaurants_preprocessed_final.csv', index=False)
print('DataFrame saved as df_restaurants_preprocessed_final.csv')

In [ ]:
from google.colab import files

# Download the file
files.download('df_restaurants_preprocessed_final.csv')

# 2 Checking for correlation between features and review score

## 2.1 Iniital correlation tests

In [ ]:
df_restaurants_model = df_restaurants.drop(columns=['latitude', 'longitude', 'review_count', 'is_open', 'business_id', 'postal_code', 'food_type', 'attire'])

We will exclude a few of the columns that we don't want to test (because they are not really features)

Open question: should we filter out restaurants that have less than some threshold of review count?

Other open question: What should we do about is_open (if anything)?

In [ ]:
# Calculate the correlation of all features with the 'stars' column
correlation_table = df_restaurants_model.corr()['stars'].round(2)

# Display the correlation table
print(correlation_table)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Calculate the correlation matrix
correlation_matrix = df_restaurants_model.corr().round(2)

# Create a heatmap with adjusted size and smaller annotation text
fig = plt.figure(figsize=(12, 10))  # Increase the figure size
sns.heatmap(correlation_matrix, cmap="coolwarm", annot=True, annot_kws={"size":8}, fmt='.2f')

# Display the plot
# plt.show()

# 3 Initial regression (without any modifications)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from sklearn.preprocessing import PolynomialFeatures

# Step 1: Prepare the data
# Ensure all boolean columns are numeric (True/False -> 1/0)
df_restaurants_model = df_restaurants_model.astype({col: 'int' for col in df_restaurants_model.select_dtypes(include=['bool']).columns})

# Handle categorical variables (if any) using one-hot encoding
df_restaurants_model = pd.get_dummies(df_restaurants_model, drop_first=True)

# Drop any rows with missing values (if not done already)
df_restaurants_model = df_restaurants_model.dropna()

# Step 2: Split the data into features (X) and target (y)
X = df_restaurants_model.drop(columns=['stars'], errors='ignore')
y = df_restaurants_model['stars']  # Target

# Add a constant to the model (intercept)
X = sm.add_constant(X)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Perform the regression using statsmodels
# Fit the model to the training data
model = sm.OLS(y_train, X_train).fit()

# Step 4: Get the summary of the model
print(model.summary())

# Step 5: Make predictions on the test set (optional)
y_pred = model.predict(X_test)

# Evaluate the model on the test set (optional)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error on Test Set: {mse}")
print(f"R-squared on Test Set: {r2}")

# 4 Improving regression

## 4.1 Checking for multicollinearity

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
import numpy as np
import pandas as pd

X_for_vif = X.drop(columns=['const'], errors='ignore')

# Ensure that the target variable 'stars' is not in the feature set
assert 'stars' not in X_for_vif.columns, "'stars' should not be in the feature set for VIF calculation."

# Initialize an empty DataFrame to store VIF results
vif_data = pd.DataFrame()
vif_data["Feature"] = X_for_vif.columns
vif_data["VIF"] = [variance_inflation_factor(X_for_vif.values, i) for i in range(X_for_vif.shape[1])]

# Display the VIF data, filtering out features with low multicollinearity (VIF < 5)
vif_filtered = vif_data[vif_data["VIF"] > 5].sort_values(by="VIF", ascending=False)
print(vif_filtered)

Let's check what these are collinear with again (correlation coefficients below):

*   open_on_weekend: 0.40 with hours_per_week
*   price_range: 0.39 with reservations, 0.37 with alcohol, -0.27 good_for_kids, 0.25 table_service, 0.25 happy hour
*   take_out: 0.33 good_for_kids, 0.29 credit_card, 0.28 delivery
*   credit_card: 0.31 good_for_groups, 0.29 take_out, 0.28 good_for_kids,
*   good_for_groups: 0.47 good_for_kids, 0.35 alcohol, 0.32 tv, 0.31 credit_card

Since we may want to keep some of these we might want to apply regularization to reduce multicollinearity issues

In [ ]:
# Create a heatmap with adjusted size and smaller annotation text
fig = plt.figure(figsize=(12, 10))  # Increase the figure size
sns.heatmap(correlation_matrix, cmap="coolwarm", annot=True, annot_kws={"size":8}, fmt='.2f')

# Display the plot
# plt.show()

## 4.2 Training random forest model to check feature importance

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Initialize the Random Forest Regressor
rf = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
rf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf.predict(X_test)

# Calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Calculate feature importances
importances = rf.feature_importances_
feature_importance = pd.DataFrame({'Feature': X.columns, 'Importance': importances})

# Sort by importance
feature_importance = feature_importance.sort_values(by='Importance', ascending=False)

# Display the feature importance
print(feature_importance)

Suggested next steps:

*  Drop lowest importance features: appointment_only, coat_check, dogs
*   Think about interaction terms for moderate_important features (0.01 to 0.1)
*   Retrain model and compare performance (using cross-validation)
*   Apply regularization

## 4.3 Testing interaction terms

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures

# Step 1: Remove any constant (intercept) before creating polynomial features
X_no_const = X.drop(columns=['const'], errors='ignore')

# Step 2: Create interaction terms using PolynomialFeatures without the constant
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
X_poly = poly.fit_transform(X_no_const)

# Get the names of the features including interaction terms
feature_names = poly.get_feature_names_out(input_features=X_no_const.columns)

# Convert the array back to a DataFrame
X_poly_df = pd.DataFrame(X_poly, columns=feature_names)

# Step 3: Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_poly_df, y, test_size=0.2, random_state=42)

# Step 4: Fit a model (e.g., RandomForest) to evaluate feature importance
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Step 5: Get the feature importance scores
importance_scores = model.feature_importances_
feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importance_scores
}).sort_values(by='Importance', ascending=False)

print(feature_importance_df)

# Step 6: Evaluate the model with interaction terms
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

In [ ]:
# Convert the Importance values to a readable format with six decimal places
feature_importance_df['Importance'] = feature_importance_df['Importance'].apply(lambda x: f'{x:.6f}')

# Replacing spaces between feature names with ' * ' to indicate interaction
feature_importance_df['Feature'] = feature_importance_df['Feature'].apply(lambda x: ' * '.join(x.split()))

# Display the formatted DataFrame
print(feature_importance_df.to_string(index=False))

## 4.4 Re-running regression after dropping some features

In [ ]:
# Dropping
X_reduced = df_restaurants_model.drop(columns=['stars', 'appointment_only', 'coat_check', 'dogs'], errors='ignore')

# Add a constant to the model (intercept)
X_reduced = sm.add_constant(X_reduced)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_reduced, y, test_size=0.2, random_state=42)

# Step 3: Perform the regression using statsmodels
# Fit the model to the training data
model = sm.OLS(y_train, X_train).fit()

# Step 4: Get the summary of the model
print(model.summary())

# Step 5: Make predictions on the test set (optional)
y_pred = model.predict(X_test)

# Evaluate the model on the test set (optional)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error on Test Set: {mse}")
print(f"R-squared on Test Set: {r2}")

In [ ]:
X_for_vif = X_reduced.drop(columns=['const'], errors='ignore')

# Ensure that the target variable 'stars' is not in the feature set
assert 'stars' not in X_for_vif.columns, "'stars' should not be in the feature set for VIF calculation."

# Initialize an empty DataFrame to store VIF results
vif_data = pd.DataFrame()
vif_data["Feature"] = X_for_vif.columns
vif_data["VIF"] = [variance_inflation_factor(X_for_vif.values, i) for i in range(X_for_vif.shape[1])]

# Display the VIF data, filtering out features with low multicollinearity (VIF < 5)
vif_filtered = vif_data[vif_data["VIF"] > 5].sort_values(by="VIF", ascending=False)
print(vif_filtered)

In [ ]:
correlation_matrix = df_restaurants_model.corr().round(2)

# Create a heatmap with adjusted size and smaller annotation text
fig = plt.figure(figsize=(12, 10))  # Increase the figure size
sns.heatmap(correlation_matrix, cmap="coolwarm", annot=True, annot_kws={"size":8}, fmt='.2f')

# Display the plot
# plt.show()

In [ ]:
# Initialize the Random Forest Regressor
rf = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
rf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf.predict(X_test)

# Calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Calculate feature importances
importances = rf.feature_importances_

# Ensure to use the feature names from the transformed dataset
feature_importance = pd.DataFrame({'Feature': X_train.columns, 'Importance': importances})

# Sort by importance
feature_importance = feature_importance.sort_values(by='Importance', ascending=False)

# Display the feature importance
print(feature_importance)

In [ ]:
# Dropping
X_reduced = df_restaurants_model.drop(columns=['stars', 'appointment_only', 'coat_check', 'dogs', 'credit_card', 'good_for_groups', 'drive_thru', 'hours_weekend'], errors='ignore')

# Creating new terms
df_restaurants_model['delivery_drive_thru'] = df_restaurants_model['delivery'] * df_restaurants_model['drive_thru']

# Add a constant to the model (intercept)
X_reduced = sm.add_constant(X_reduced)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_reduced, y, test_size=0.2, random_state=42)

# Step 3: Perform the regression using statsmodels
# Fit the model to the training data
model = sm.OLS(y_train, X_train).fit()

# Step 4: Get the summary of the model
print(model.summary())

# Step 5: Make predictions on the test set (optional)
y_pred = model.predict(X_test)

# Evaluate the model on the test set (optional)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error on Test Set: {mse}")
print(f"R-squared on Test Set: {r2}")

In [ ]:
X_for_vif = X_reduced.drop(columns=['const'], errors='ignore')

# Ensure that the target variable 'stars' is not in the feature set
assert 'stars' not in X_for_vif.columns, "'stars' should not be in the feature set for VIF calculation."

# Initialize an empty DataFrame to store VIF results
vif_data = pd.DataFrame()
vif_data["Feature"] = X_for_vif.columns
vif_data["VIF"] = [variance_inflation_factor(X_for_vif.values, i) for i in range(X_for_vif.shape[1])]

# Display the VIF data, filtering out features with low multicollinearity (VIF < 5)
vif_filtered = vif_data[vif_data["VIF"] > 5].sort_values(by="VIF", ascending=False)
print(vif_filtered)

In [ ]:
# Initialize the Random Forest Regressor
rf = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
rf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf.predict(X_test)

# Calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Calculate feature importances
importances = rf.feature_importances_

# Ensure to use the feature names from the transformed dataset
feature_importance = pd.DataFrame({'Feature': X_train.columns, 'Importance': importances})

# Sort by importance
feature_importance = feature_importance.sort_values(by='Importance', ascending=False)

# Display the feature importance
print(feature_importance)

### Let's try Lasso and Ridge

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, Ridge
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

# Step 1: Prepare the data (you've already done this part)
X_reduced = df_restaurants_model.drop(columns=['stars', 'appointment_only', 'coat_check', 'dogs', 'credit_card', 'good_for_groups', 'open_on_weekend', 'drive_thru'], errors='ignore')

# Create the new interaction term
df_restaurants_model['delivery_drive_thru'] = df_restaurants_model['delivery'] * df_restaurants_model['drive_thru']
df_restaurants_model['hours_weekend'] = df_restaurants_model['hours_per_week'] * df_restaurants_model['open_on_weekend']

# Step 2: Standardize the features
scaler = StandardScaler()
X_reduced_scaled = scaler.fit_transform(X_reduced)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_reduced_scaled, y, test_size=0.2, random_state=42)

# Step 3: Fit Lasso Regression
lasso = Lasso(alpha=0.1, random_state=42)  # You can tune the alpha parameter
lasso.fit(X_train, y_train)

# Step 4: Fit Ridge Regression
ridge = Ridge(alpha=1.0, random_state=42)  # You can tune the alpha parameter
ridge.fit(X_train, y_train)

# Step 5: Evaluate the Lasso model
y_pred_lasso = lasso.predict(X_test)
mse_lasso = mean_squared_error(y_test, y_pred_lasso)
r2_lasso = r2_score(y_test, y_pred_lasso)

print("Lasso Regression Results")
print(f"Mean Squared Error: {mse_lasso}")
print(f"R-squared: {r2_lasso}")

# Step 6: Evaluate the Ridge model
y_pred_ridge = ridge.predict(X_test)
mse_ridge = mean_squared_error(y_test, y_pred_ridge)
r2_ridge = r2_score(y_test, y_pred_ridge)

print("\nRidge Regression Results")
print(f"Mean Squared Error: {mse_ridge}")
print(f"R-squared: {r2_ridge}")

In [ ]:
X_for_vif = X_reduced.drop(columns=['const'], errors='ignore')

# Ensure that the target variable 'stars' is not in the feature set
assert 'stars' not in X_for_vif.columns, "'stars' should not be in the feature set for VIF calculation."

# Initialize an empty DataFrame to store VIF results
vif_data = pd.DataFrame()
vif_data["Feature"] = X_for_vif.columns
vif_data["VIF"] = [variance_inflation_factor(X_for_vif.values, i) for i in range(X_for_vif.shape[1])]

# Display the VIF data, filtering out features with low multicollinearity (VIF < 5)
vif_filtered = vif_data[vif_data["VIF"] > 5].sort_values(by="VIF", ascending=False)
print(vif_filtered)

## 4.6 Using recursive feature elimination

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

model = LinearRegression()
rfe = RFE(model, n_features_to_select=10)
rfe = rfe.fit(X_train, y_train)

print(f"Selected Features: {X_train.columns[rfe.support_]}")

In [ ]:
selected_features = X_train.columns[rfe.support_]
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

# Retrain the model with selected features
model.fit(X_train_selected, y_train)
y_pred_selected = model.predict(X_test_selected)

# Evaluate the model
mse_selected = mean_squared_error(y_test, y_pred_selected)
r2_selected = r2_score(y_test, y_pred_selected)

print(f"Mean Squared Error with Selected Features: {mse_selected}")
print(f"R-squared with Selected Features: {r2_selected}")

In [ ]:
from sklearn.model_selection import cross_val_score

cv_scores = cross_val_score(model, X_train_selected, y_train, cv=5)
print(f"Cross-Validation Scores with Selected Features: {cv_scores}")

# 5 Testing different **models**

Things to try: SGD regressor, k-neighbors (run grid search -> penalty and alpha), random forest, gradient-boosting regressor, xgboost, svm

## 5.1 SGD regressor

In [ ]:
import numpy as np
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error, r2_score

First we run a grid search to optimize parameters

In [ ]:
param_grid = {
    'alpha': [0.0001, 0.001, 0.01, 0.1],
    'penalty': ['l2', 'l1', 'elasticnet'],
    'max_iter': [1000, 2000, 5000],
    'learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive'],
    'eta0': [0.01, 0.1, 1.0]
}

We want to do this for X_reduced first

In [ ]:
# Split your data into training and testing sets
X_train_red, X_test_red, y_train_red, y_test_red = train_test_split(X_reduced, y, test_size=0.2, random_state=42)

# Initialize the SGDRegressor
sgd = SGDRegressor(random_state=42)

# Set up the grid search
grid_search_red = GridSearchCV(sgd, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit the model on the reduced dataset
grid_search_red.fit(X_train_red, y_train_red)

# Get the best model and parameters
best_model_red = grid_search_red.best_estimator_
print("Best parameters for X_reduced:", grid_search_red.best_params_)

# Evaluate the model
y_pred_red = best_model_red.predict(X_test_red)
mse_red = mean_squared_error(y_test_red, y_pred_red)
r2_red = r2_score(y_test_red, y_pred_red)

print(f"X_reduced - Mean Squared Error: {mse_red}")
print(f"X_reduced - R-squared: {r2_red}")

And then X_selected (from recursive feature selection)

In [ ]:
# Split your data into training and testing sets
X_train_sel, X_test_sel, y_train_sel, y_test_sel = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# Initialize the SGDRegressor
sgd = SGDRegressor(random_state=42)

# Set up the grid search
grid_search_sel = GridSearchCV(sgd, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit the model on the selected dataset
grid_search_sel.fit(X_train_sel, y_train_sel)

# Get the best model and parameters
best_model_sel = grid_search_sel.best_estimator_
print("Best parameters for X_selected:", grid_search_sel.best_params_)

# Evaluate the model
y_pred_sel = best_model_sel.predict(X_test_sel)
mse_sel = mean_squared_error(y_test_sel, y_pred_sel)
r2_sel = r2_score(y_test_sel, y_pred_sel)

print(f"X_selected - Mean Squared Error: {mse_sel}")
print(f"X_selected - R-squared: {r2_sel}")

## 5.2 K-neighbors

## 5.3 Random forest

## 5.4 Gradient-boosting regressor

## 5.5 Xgboost

## 5.6 SVM

## 5.7 Optimize across models